In [136]:
import pandas as pd
from sklearn import preprocessing
!pip install xlrd
!pip install openpyxl

In [137]:
gdp_df = pd.read_csv('datasets/GDP1.csv')
### Next line unpivots table to have each year on a single row
gdp_df = pd.melt(gdp_df, id_vars='Country Name', value_vars=list(gdp_df.columns[4:]), var_name='Year', value_name='GDP')
gdp_df = gdp_df.rename(columns={'Country Name': 'Country'})
gdp_df['Year'] = gdp_df['Year'].astype('int64')

In [138]:
# contains all other info about how happiness score was derived. From World Happiness Report
happy_df = pd.read_excel('datasets/DataForTable2.1.xls')
happy_df = happy_df.rename(columns={'Country name': 'Country', 'year': 'Year', 'Life Ladder': 'Happiness Score'})

In [139]:
le_df = pd.read_csv('datasets/Life_Expectancy.csv')
### Next line unpivots table to have each year on a single row
le_df = pd.melt(le_df, id_vars='Country Name', value_vars=list(le_df.columns[4:]), var_name='Year', value_name='Life Expectancy')
le_df = le_df.rename(columns={'Country Name': 'Country'})
le_df['Year'] = le_df['Year'].astype('int64')

In [140]:
xlsx = pd.ExcelFile('datasets/Freedom.xlsx')
freedom_df = pd.read_excel(xlsx, 'FIW13-22', header=1)
freedom_df = freedom_df.rename(columns={'Country/Territory': 'Country', 'Edition': 'Year', 'Total': 'Freedom_Rating'})
freedom_df = freedom_df[['Country', 'Year', 'Freedom_Rating']]

In [147]:
# merge all dfs
df = happy_df.merge(freedom_df, on=['Country', 'Year'], how='inner')
df = df.merge(le_df, on=['Country', 'Year'], how='inner')
df = df.merge(gdp_df, on=['Country', 'Year'], how='inner')

In [142]:
df.to_csv('all_data.csv')

In [152]:
test = df.copy()
# Divide happiness scores into percentiles based on year
test['Happiness_Score_Percentile'] = df.groupby('Year')['Happiness Score'].transform(lambda x: pd.cut(x, bins=10, labels=[9,8,7,6,5,4,3,2,1,0]))
test['Percentile Ranges'] = df.groupby('Year')['Happiness Score'].transform(lambda x: pd.cut(x, bins=10))

In [149]:
normalized = test.copy()
numeric_columns = normalized.select_dtypes(include=['number']).columns
numeric_columns = numeric_columns[2:]
# fill NaN with country's average
normalized[numeric_columns] = normalized[numeric_columns].fillna(normalized.groupby('Country')[numeric_columns].transform('mean'))
# Mean normalize each factor
normalized[numeric_columns] = normalized.groupby('Year')[numeric_columns].transform(lambda x: (x - x.mean()) / x.std())

In [145]:
normalized.to_csv('normalized_data.csv')